### Homework 1
### Ex 1

In [1]:
import torch
import torchvision
from torchvision import transforms,datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [19]:
X1 = np.array(np.mat('1 0;-1 0; 0 1;0 -1'))
X1 = torch.from_numpy(X1).type(torch.FloatTensor)
Y1 = np.array([0,0,1,1])
Y1 = torch.from_numpy(Y1).type(torch.LongTensor)

In [20]:
df = pd.read_csv('data_HW2_ex1.csv')
df = df.sample(frac=1).reset_index(drop=True)
X = np.column_stack((df['x1'].values,df['x2'].values))
size = int(0.8*len(X))
y = df['class'].values
Y = np.where(y==1, 0, y)
Y = np.where(y==2, 1, Y)
X_train = X[:size]
X_test = X[size:]
Y_train = Y[:size]
Y_test = Y[size:]
print(Y_test)
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
Y_train = torch.from_numpy(Y_train).type(torch.LongTensor)
Y_test = torch.from_numpy(Y_test).type(torch.LongTensor)    

[1 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 1 0 1 1
 1 1 1]


### Data Loading

In [27]:
from torch.utils.data import Dataset, DataLoader

class Dataset(Dataset):
    def __init__(self, datapt,labels):
        'Initialization'
        self.X = datapt
        self.y = labels

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.X)

    def __getitem__(self, i):
        'Generates one sample of data'
        # Select sample
        data = self.X[i]
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

params = {'batch_size': 10,'shuffle': True}
dataset1 = Dataset(X1,Y1)
data1 = DataLoader(dataset1)
traindataset = Dataset(X_train,Y_train)
training_data = DataLoader(traindataset, **params)
testdataset = Dataset(X_test,Y_test)
test_data = DataLoader(testdataset, **params)

### Defining the Neural Network

In [33]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2,2)
        self.fc2 = nn.Linear(2,2)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
net = Net()

### Training the Network (Problem 1(a))

In [34]:
import torch.optim as optim

optimizer = torch.optim.Adam(net.parameters(),lr = 0.001)

epoch = 100
for epoch in range(epoch):
    for data in data1:
        X,y = data
        optimizer.zero_grad()
        #x = X.view(-1,2)
        output = net(X)
        #print(output)
        loss = F.nll_loss(output,y)  
        loss.backward()
        optimizer.step()
print("Overall loss:",loss)

C:\Users\Mohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Overall loss: tensor(0.7611, grad_fn=<NllLossBackward>)


### Training the Network (Problem 1(b))

In [35]:

optimizer = torch.optim.Adam(net.parameters(),lr = 0.001)

epoch = 100
for epoch in range(epoch):
    for data in training_data:
        X,y = data
        optimizer.zero_grad()
        #x = X.view(-1,2)
        output = net(X)
        #print(output)
        loss = F.nll_loss(output,y)  
        loss.backward()
        optimizer.step()
print("Overall Loss:",loss)

C:\Users\Mohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Overall Loss: tensor(0.5343, grad_fn=<NllLossBackward>)


### Model Accuracy for 1(a)
### Weights and Biases of Network 1(a)

In [36]:
correct = 0
total = 0
with torch.no_grad():
    for data in data1:
        X,y = data
        output = net(X.view(-1,2))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct+=1
            total+=1
print("Model Accuracy:",correct/total*100)
print(net.state_dict())

C:\Users\Mohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Model Accuracy: 75.0
OrderedDict([('fc1.weight', tensor([[-1.3728,  0.1284],
        [-1.4411,  0.1257]])), ('fc1.bias', tensor([-0.2929, -0.3210])), ('fc2.weight', tensor([[ 1.2232,  0.3289],
        [-0.7923, -1.3425]])), ('fc2.bias', tensor([0.0250, 0.5383]))])


### Model Accuracy 1(b)
### Weights and Biases for Network 1(b)

In [37]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_data:
        X,y = data
        output = net(X.view(-1,2))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct+=1
            total+=1
print("Model Accuracy:",correct/total*100)
print(net.state_dict())

C:\Users\Mohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Model Accuracy: 67.5
OrderedDict([('fc1.weight', tensor([[-1.3728,  0.1284],
        [-1.4411,  0.1257]])), ('fc1.bias', tensor([-0.2929, -0.3210])), ('fc2.weight', tensor([[ 1.2232,  0.3289],
        [-0.7923, -1.3425]])), ('fc2.bias', tensor([0.0250, 0.5383]))])
